In [1]:
import pandas as pd
from langdetect import detect, detect_langs, DetectorFactory
import re

In [2]:
# Znovu parsovat stranky a zistovat ci su znaky z anglickej abecedy. Ak nie, tak disregardnut celu stranku?

# useful commands
# df.apply(lambda x : x.....) - vykona funkciu pre cely stlpec po jednotlivych prvkoch
# sentence to vec stanleyfok
# tf-idf - ngram-frazy
# page rank

# Filtering and preparing data

In [3]:
# Dataset so zlucenymi hodnotami
df = pd.read_csv("dataset_final_concat.csv")
# df = df.loc[df['type'].isin(['Education', 'Company'])]
df.head(50)

,index,name,link,value,type
0,0,Australian National University,http://www.anu.edu.au/,ANU Our COVID-19 pathway b' html \n [if l...,Education
1,1,Monash University,http://www.monash.edu/,Monash University - one of the top universitie...,Education
2,2,University of Queensland,http://www.uq.edu.au/,The University of Queensland \n Stu...,Education
3,3,Macquarie University,http://mq.edu.au/,\n \n\n Macquarie Universi...,Education
4,4,UNSW Sydney,https://www.unsw.edu.au/,UNSW Sydney | One of the best universities in ...,Education
5,5,Newcastle University,http://www.ncl.ac.uk/,The things we do here make a difference out th...,Education
6,6,University of Wollongong,https://www.uow.edu.au/index.html,Home - University of Wollongong \xe2\x80\x93 U...,Education
7,7,University of Melbourne,http://www.unimelb.edu.au/,"The University of Melbourne, Australia - Austr...",Education
8,8,University of Tasmania,http://www.utas.edu.au/,University of Tasmania | University of Tasmani...,Education
9,9,University of Adelaide,http://www.adelaide.edu.au/,The University of Adelaide The University of A...,Education


In [4]:
# Remove linebreaks (\n) from strings
df['value'] = df['value'].apply(lambda x: x.replace("\\n", ""))
df.head()

,index,name,link,value,type
0,0,Australian National University,http://www.anu.edu.au/,ANU Our COVID-19 pathway b' html [if lt ...,Education
1,1,Monash University,http://www.monash.edu/,Monash University - one of the top universitie...,Education
2,2,University of Queensland,http://www.uq.edu.au/,The University of Queensland Study...,Education
3,3,Macquarie University,http://mq.edu.au/,Macquarie University | T...,Education
4,4,UNSW Sydney,https://www.unsw.edu.au/,UNSW Sydney | One of the best universities in ...,Education


In [5]:
# Remove everything between <>
df['value'] = df['value'].apply(lambda x: re.sub("<.*?>", "", x))
df.head()

,index,name,link,value,type
0,0,Australian National University,http://www.anu.edu.au/,ANU Our COVID-19 pathway b' html [if lt ...,Education
1,1,Monash University,http://www.monash.edu/,Monash University - one of the top universitie...,Education
2,2,University of Queensland,http://www.uq.edu.au/,The University of Queensland Study...,Education
3,3,Macquarie University,http://mq.edu.au/,Macquarie University | T...,Education
4,4,UNSW Sydney,https://www.unsw.edu.au/,UNSW Sydney | One of the best universities in ...,Education


In [6]:
# Downloading punkt to resolve errors
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Martin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
# Remove URL links
# URL patter with protocol
url_pattern1 = "^\"?https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)\"?$"
# URL patter without protocol
url_pattern2 = "^[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)$"

df['value'] = df['value'].apply(lambda x: " ".join([w for w in x.split() if not (re.match(url_pattern1, w) or re.match(url_pattern2, w))]))
df.head(100)

,index,name,link,value,type
0,0,Australian National University,http://www.anu.edu.au/,ANU Our COVID-19 pathway b' html [if lt IE 7 ]...,Education
1,1,Monash University,http://www.monash.edu/,Monash University - one of the top universitie...,Education
2,2,University of Queensland,http://www.uq.edu.au/,The University of Queensland Study at UQ Find ...,Education
3,3,Macquarie University,http://mq.edu.au/,Macquarie University | Tertiary Education Macq...,Education
4,4,UNSW Sydney,https://www.unsw.edu.au/,UNSW Sydney | One of the best universities in ...,Education
...,...,...,...,...,...
95,108,European Molecular Biology Laboratory,http://embl.org/,European Molecular Biology Laboratory | b' htm...,Government
96,109,PSA Peugeot Citroën (France),http://www.psa-peugeot-citroen.com/en,Official Global Website | Stellantis Menu Stel...,Company
97,110,London South Bank University,http://www.lsbu.ac.uk/,London South Bank University | London South Ba...,Education
98,111,Physikalisch-Technische Bundesanstalt,https://www.ptb.de/cms/,Physikalisch-Technische Bundesanstalt Die PTB ...,Facility


In [8]:
# Remove special characters with regex
df['value'] = df['value'].apply(lambda x: re.sub(r'[^A-Za-z0-9 ]+', '', str(x)))
df.head()

,index,name,link,value,type
0,0,Australian National University,http://www.anu.edu.au/,ANU Our COVID19 pathway b html if lt IE 7 end...,Education
1,1,Monash University,http://www.monash.edu/,Monash University one of the top universities...,Education
2,2,University of Queensland,http://www.uq.edu.au/,The University of Queensland Study at UQ Find ...,Education
3,3,Macquarie University,http://mq.edu.au/,Macquarie University Tertiary Education Macqu...,Education
4,4,UNSW Sydney,https://www.unsw.edu.au/,UNSW Sydney One of the best universities in A...,Education


In [9]:
# Remove short characters and change words to lower case
df['value'] = df['value'].apply(lambda x: " ".join([w.lower() for w in x.split() if len(w) > 2]))
df.head(50)

,index,name,link,value,type
0,0,Australian National University,http://www.anu.edu.au/,anu our covid19 pathway html endif skip naviga...,Education
1,1,Monash University,http://www.monash.edu/,monash university one the top universities aus...,Education
2,2,University of Queensland,http://www.uq.edu.au/,the university queensland study find degree st...,Education
3,3,Macquarie University,http://mq.edu.au/,macquarie university tertiary education macqua...,Education
4,4,UNSW Sydney,https://www.unsw.edu.au/,unsw sydney one the best universities australi...,Education
5,5,Newcastle University,http://www.ncl.ac.uk/,the things here make difference out there newc...,Education
6,6,University of Wollongong,https://www.uow.edu.au/index.html,home university wollongong xe2x80x93 uow study...,Education
7,7,University of Melbourne,http://www.unimelb.edu.au/,the university melbourne australia australias ...,Education
8,8,University of Tasmania,http://www.utas.edu.au/,university tasmania university tasmania browse...,Education
9,9,University of Adelaide,http://www.adelaide.edu.au/,the university adelaide the university adelaid...,Education


In [10]:
# Downloading stopwords to resolve errors
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Martin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
# Stop word removal with nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words.update(["html", "endif"])

df['value'] = df['value'].apply(lambda x: " ".join([word for word in word_tokenize(x) if word not in stop_words]))
df.head()

,index,name,link,value,type
0,0,Australian National University,http://www.anu.edu.au/,anu covid19 pathway skip navigation,Education
1,1,Monash University,http://www.monash.edu/,monash university one top universities austral...,Education
2,2,University of Queensland,http://www.uq.edu.au/,university queensland study find degree study ...,Education
3,3,Macquarie University,http://mq.edu.au/,macquarie university tertiary education macqua...,Education
4,4,UNSW Sydney,https://www.unsw.edu.au/,unsw sydney one best universities australia ex...,Education


In [13]:
# # stem the words
# from nltk.stem.porter import PorterStemmer
# stemmer = PorterStemmer()
#
# df['value'] = df['value'].apply(lambda sentence : " ".join([stemmer.stem(word) for word in sentence.split(" ")]))
# df.head()

In [14]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Martin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
# OR Lemmatize words
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

df['value'] = df['value'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in x.split()]))
df.head()

,index,name,link,value,type
0,0,Australian National University,http://www.anu.edu.au/,anu covid19 pathway skip navigation,Education
1,1,Monash University,http://www.monash.edu/,monash university one top university australia...,Education
2,2,University of Queensland,http://www.uq.edu.au/,university queensland study find degree study ...,Education
3,3,Macquarie University,http://mq.edu.au/,macquarie university tertiary education macqua...,Education
4,4,UNSW Sydney,https://www.unsw.edu.au/,unsw sydney one best university australia expl...,Education


In [16]:
# Removing rows, where 'value' is nan
print(df['value'].isna().sum())
print(df.shape)

df = df[df['value'].notna()]

print(df['value'].isna().sum())
print(df.shape)

0
(83796, 5)
0
(83796, 5)


In [17]:
# Remove empty rows
print(df['value'].isna().sum())
print(df.shape)

df = df[df['value'] != ""]

print(df['value'].isna().sum())
print(df.shape)

0
(83796, 5)
0
(82457, 5)


In [18]:
# Removing rows where language is not English
def language_detection(text, method = "single"):
    try:
        result = detect_langs(text) if method.lower() != "single" else detect(text)
        return result == "en"
    except:
        print(text)
        return False

DetectorFactory.seed = 0
print(language_detection("Hello there, what a lovely evening!"))
df = df[df['value'].apply(language_detection)]
print(df.shape)

True
74862272
99745
(49555, 5)


In [19]:
df

,index,name,link,value,type
0,0,Australian National University,http://www.anu.edu.au/,anu covid19 pathway skip navigation,Education
1,1,Monash University,http://www.monash.edu/,monash university one top university australia...,Education
2,2,University of Queensland,http://www.uq.edu.au/,university queensland study find degree study ...,Education
3,3,Macquarie University,http://mq.edu.au/,macquarie university tertiary education macqua...,Education
4,4,UNSW Sydney,https://www.unsw.edu.au/,unsw sydney one best university australia expl...,Education
...,...,...,...,...,...
83787,101248,Erasmus University Rotterdam,http://www.eur.nl/english/,erasmus university rotterdam erasmus universit...,Education
83788,101249,LifeStream,https://www.lstream.org/,lifestream blood bank type blood needed platel...,Nonprofit
83790,101252,Shirak State University,http://shsu.am/en/,xc2xabm nalbandyan state university shirakxc2x...,Education
83793,101255,Radboud University Nijmegen,http://www.ru.nl/english/,radboud university main menu discover programm...,Education


In [20]:
df.shape
df.to_csv("dataset_final_lemmatize.csv")